In [2]:
import sys
sys.path.append("../..")

In [3]:
from src.marks_reader import *
from src.competition_data import *
from src.optimizer import *
from src.prediction_saver import *
from events_data import events
from datetime import datetime

In [5]:
events_params = load_json('opt_results.json')

In [4]:
competition = CompetitionData(
    name="World Championships in Athletics Doha 2019", 
    events_data=events,
    events_params=events_params,
    start_date=datetime(2019, 9, 27),
)

event_folder = 'classified'
only = None

In [5]:
data = read_marks(
    folder_path=event_folder, 
    only=only,
    limit_date=competition.start_date
)

In [6]:
results = load_json('results.json')

In [7]:
def save_config(config: dict, event: str, sex: str, file: str = 'opt_results.json') -> None:
    optimal_results = load_json(file)
    
    if not event in optimal_results:
        optimal_results[event] = {}

    if not sex in optimal_results[event]:
        optimal_results[event][sex] = {}
    
    for key, value in config.items():
        optimal_results[event][sex][key] = value

    save_json(optimal_results, file)

In [8]:
def optimize_params(events: list, data: dict, results: dict, competition: CompetitionData, params: list):
    
    for event in events:
        for sex in competition.get_event_data(event, 'sex'):
            print(f"Optimizing {event} - {sex}...")

            optimizer = EventOptimizer(
                runcount=10,
                calculate_error=calculate_error3,
                optimize_params=params,
                default_params={
                    "bandwidth": competition.get_event_param(event, sex, 'bandwidth', 1),
                    "alpha": competition.get_event_param(event, sex, 'alpha', 0),
                    "sim_times": competition.get_event_param(event, sex, 'sim_times', 5000),
                    "pond_times": {competition.start_date.year - i : int(4 / (i+1)) for i in range(3)},
                }
            )

            optimal_config = optimizer.optimize(data[event][sex], list(results[event][sex].values()), event, sex, competition)
            print(optimal_config)
            save_config(optimal_config, event, sex)

In [9]:
optimize_params(only, data, results, competition, ['sim_times'])

Optimizing atl_alt - male...
Configuration(values={
  'sim_times': '3000',
})

Optimizing atl_alt - female...
Configuration(values={
  'sim_times': '9000',
})

Optimizing atl_3000m - male...
Configuration(values={
  'sim_times': '6000',
})

Optimizing atl_3000m - female...
Configuration(values={
  'sim_times': '10000',
})

Optimizing atl_400v - male...
Configuration(values={
  'sim_times': '7000',
})

Optimizing atl_400v - female...
Configuration(values={
  'sim_times': '4000',
})

Optimizing atl_110v - male...
Configuration(values={
  'sim_times': '10000',
})

Optimizing atl_100v - female...
Configuration(values={
  'sim_times': '1000',
})

Optimizing atl_10000m - male...
Configuration(values={
  'sim_times': '1000',
})

Optimizing atl_10000m - female...
Configuration(values={
  'sim_times': '8000',
})

Optimizing atl_5000m - male...
Configuration(values={
  'sim_times': '9000',
})

Optimizing atl_5000m - female...
Configuration(values={
  'sim_times': '8000',
})

Optimizing atl_1500m

In [8]:
optmizer = PondYearsOptimizer(512, calculate_error3) # Solo se ejecuta 120 veces
optimal_configs = optmizer.optimize(data, results, competition)
print(optimal_configs)

{2021: 6, 2020: 4, 2019: 3}
2932
{2021: 4, 2020: 4, 2019: 4}
3071
{2021: 8, 2020: 8, 2019: 5}
3101
{2021: 7, 2020: 5, 2019: 4}
3296
{2021: 8, 2020: 3, 2019: 2}


KeyboardInterrupt: 